In [1]:
class MyMeta(type):
    def __new__(meta, name, bases, namespace):
        # for attr in namespace.get("__readonly__", []):
        #     value = namespace.get(attr)
        #     namespace[attr] = property(lambda self: value)

        cls = super().__new__(meta, name, bases, namespace)

        readonly = set()
        for base in reversed(cls.__mro__):
            readonly.update(getattr(base, "__readonly__", []))

        for attr in readonly:
            value = namespace[attr] if attr in namespace else getattr(cls, attr, None)
            setattr(cls, attr, property(lambda self: value))

        cls.__readonly__ = readonly
        return cls
    
    # def __setattr__(cls, name, value):
    #     if name in cls.__readonly__:
    #         raise AttributeError(f"{name} is a readonly attribute.")
    #     super().__setattr__(name, value)


class A(metaclass=MyMeta):
    __readonly__ = ["foo"]
    
    foo = 42

class B(A):
    __readonly__ = ["bar"]
    
    bar = "hello there"

class C:
    pass

class D(C, B):
    __readonly__ = ["gaz"]
    
    gaz = 0

b = B()

b.__readonly__

{'bar', 'foo'}

In [2]:
D.__readonly__

{'bar', 'foo', 'gaz'}

In [3]:
b.bar

'hello there'

In [4]:
try:
    b.bar = "hello"
except Exception:
    print("can't modify 'bar' attribute")

can't modify 'bar' attribute


In [5]:
b.bar

'hello there'

In [6]:
B.bar

In [7]:
try:
    B.bar = "hello"
except Exception:
    print("can't modify 'bar' attribute")

In [8]:
try:
    setattr(B, "bar", "hello")
except Exception:
    print("can't modify 'bar' attribute")

In [9]:
try:
    object.__setattr__(B, "bar", "hello")
except Exception:
    print("can't modify 'bar' attribute")

can't modify 'bar' attribute


In [10]:
b.bar

'hello'

In [11]:
b.__dict__

{}

In [12]:
b.__dict__["bar"] = "hello"

In [13]:
b.bar

'hello'

In [14]:
b.__dict__["gaz"] = 0

In [15]:
b.__dict__

{'bar': 'hello', 'gaz': 0}